In [1]:
"""
Extract TreeMap (c.a. 2016) statistics in active fire detections

"""

import os, sys
import ee
import geemap
import time
import pandas as pd

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()

ee.Initialize(project='jfsp-aspen')

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Success")

Success


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [2]:
# Load the USFS TreeMap
treemap = ee.ImageCollection("USFS/GTAC/TreeMap/v2016")
print(f"TreeMap bands available for analysis:\n\n{treemap.first().bandNames().getInfo()}")

TreeMap bands available for analysis:

['ALSTK', 'BALIVE', 'CANOPYPCT', 'CARBON_D', 'CARBON_DWN', 'CARBON_L', 'DRYBIO_D', 'DRYBIO_L', 'FLDSZCD', 'FLDTYPCD', 'FORTYPCD', 'GSSTK', 'QMD_RMRS', 'SDIPCT_RMRS', 'STANDHT', 'STDSZCD', 'TPA_DEAD', 'TPA_LIVE', 'Value', 'VOLBFNET_L', 'VOLCFNET_D', 'VOLCFNET_L']


In [10]:
class_codes = treemap.first().get('FORTYPCD_class_values').getInfo()
class_names = treemap.first().get('FORTYPCD_class_names').getInfo()
code_to_name = dict(zip(class_codes, class_names))
species_df = pd.DataFrame(list(code_to_name.items()), columns=['FORTYPCD', 'SpeciesName'])
species_df['FORTYPCD'] = species_df['FORTYPCD'].astype(int)
species_df = species_df.reset_index(drop=True)
species_df.head()

,FORTYPCD,SpeciesName
0,101,Jack pine
1,102,Red pine
2,103,Eastern white pine
3,104,Eastern white pine / eastern hemlock
4,105,Eastern hemlock


In [11]:
# Save this file out.
out_fp = os.path.join(projdir,'data/tabular/mod/treemap_fortypcd_species_mapping.csv')
species_df.to_csv(out_fp)
print(f"Dictionary saved to {out_fp}")

Dictionary saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/tabular/mod/treemap_fortypcd_species_mapping.csv


In [ ]:
# Load the Active Fire Detections (AFD)
afds = ee.FeatureCollection('projects/jfsp-aspen/assets/AFD/vnp14img_geo_srm_pix_area_aspenfires')
print(afds.size().getInfo())
print(afds.first().propertyNames().getInfo())

In [ ]:
# add a small buffer to the AFDs


## Calculate species-specific cover, live basal area (mean), and canopy percent (mean)

In [ ]:
# Clean the property names before exporting reductions (just keep the ID)
afds_ = afds.select(['afdID'])

In [ ]:
def species_histogram(ftr):
    image = treemap.select('FORTYPCD').first()
    hist = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=ftr.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('FORTYPCD')

    hist_dict = ee.Dictionary(hist)
    flat_ftr = hist_dict.keys().iterate(
        lambda key, f: ee.Feature(f).set(ee.String(key), hist_dict.get(key)), ftr
    )
    return ee.Feature(flat_ftr)


# Apply this to the AFDs
fortypcd = afds_.map(species_histogram)

# Print a sample
sample = fortypcd.limit(1).getInfo()
properties = sample['features'][0]['properties']
df = pd.DataFrame([properties])
print("Sample DataFrame:")
print(df.head())

# Export to Asset/Drive
fortypcd = fortypcd.map(lambda ftr: ftr.setGeometry(None)) # drop geometry column

task = ee.batch.Export.table.toDrive(
    collection=fortypcd,
    description='vnp14img_treemap-fortypcd',
    fileFormat='CSV',
    folder='TreeMap'
)

# Start the export task
task.start()
print("Export to Google Drive started!")
# Monitor the task until it's finished
monitor_export(task, timeout=120) # print every X seconds

In [ ]:
# Species BALIVE and CANOPYPCT

In [ ]:
def species_metrics(ftr):
    image = treemap.mosaic()
    # Get species histogram
    hist = image.select('FORTYPCD').reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=ftr.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('FORTYPCD')

    hist_dict = ee.Dictionary(hist)
    
    # Initialize feature with species histogram keys and calculate averages
    def add_species_metrics(key, f):
        key_str = ee.String(key)
        # Mask the image by the species type
        masked_image = image.updateMask(image.select('FORTYPCD').eq(ee.Number.parse(key).toInt()))
        
        # Calculate average BALIVE and CANOPYPCT for the masked image within the polygon
        metrics = masked_image.select(['BALIVE', 'CANOPYPCT']).reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=ftr.geometry(),
            scale=30,
            maxPixels=1e13
        )
        
        # Add the averages to the feature with species-specific prefix
        f = ee.Feature(f).set(key_str.cat('_BALIVE_mean'), metrics.get('BALIVE'))
        f = f.set(key_str.cat('_CANOPYPCT_mean'), metrics.get('CANOPYPCT'))
        return f

    # Apply species metrics calculation to each species type in histogram
    flat_ftr = hist_dict.keys().iterate(add_species_metrics, ftr)
    
    return ee.Feature(flat_ftr)

fortypcd_metrics = afds_.map(species_metrics)

# Remove geometry column first
fortypcd_metrics = fortypcd_metrics.map(lambda ftr: ftr.setGeometry(None)) # drop geometry column

export_task = ee.batch.Export.table.toDrive(
    collection=fortypcd_metrics,
    description='vnp14img_treemap-balive_canopypct',
    fileFormat='CSV', 
    fileNamePrefix='vnp14img_treemap-balive_canopypct',
    folder='TreeMap'
)

# Start the export task
export_task.start()
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_export(export_task, 120) # print every 120 seconds